## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-24-13-02-40 +0000,nyt,Social Security Announces a 2.8% Cost-of-Livin...,https://www.nytimes.com/2025/10/24/business/so...
1,2025-10-24-13-01-00 +0000,wsj,Alaska Airlines Works to Restore Operations Af...,https://www.wsj.com/business/airlines/alaska-a...
2,2025-10-24-13-00-09 +0000,nyt,What to know about the report.,https://www.nytimes.com/live/2025/10/24/busine...
3,2025-10-24-13-00-07 +0000,nyt,Comparing Savings Accounts and CDs Before the ...,https://www.nytimes.com/2025/10/24/business/in...
4,2025-10-24-13-00-06 +0000,wapo,Rubio warns there is ‘no plan B’ as Trump’s Ga...,https://www.washingtonpost.com/world/2025/10/2...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
32,trump,65
118,off,16
325,new,16
121,canada,13
279,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
158,2025-10-24-03-21-49 +0000,cbc,Trump claims Canada 'CHEATED AND GOT CAUGHT' w...,https://www.cbc.ca/news/world/trump-trade-nego...,149
23,2025-10-24-12-13-55 +0000,cbc,LIVE UPDATES | Trump says he’s terminating all...,https://www.cbc.ca/news/canada/livestory/live-...,146
204,2025-10-23-23-29-06 +0000,nypost,Ford warns devasting plant fire will hurt prof...,https://nypost.com/2025/10/23/business/ford-wa...,132
110,2025-10-24-08-25-06 +0000,wapo,Trump says he’s canceling trade negotiations w...,https://www.washingtonpost.com/world/2025/10/2...,130
159,2025-10-24-03-16-11 +0000,nypost,Trump terminates trade talks with Canada over ...,https://nypost.com/2025/10/23/us-news/trump-te...,129


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
158,149,2025-10-24-03-21-49 +0000,cbc,Trump claims Canada 'CHEATED AND GOT CAUGHT' w...,https://www.cbc.ca/news/world/trump-trade-nego...
204,67,2025-10-23-23-29-06 +0000,nypost,Ford warns devasting plant fire will hurt prof...,https://nypost.com/2025/10/23/business/ford-wa...
239,61,2025-10-23-21-24-00 +0000,wsj,Lawmakers failed to provide a last-minute repr...,https://www.wsj.com/politics/policy/air-traffi...
288,53,2025-10-23-18-09-53 +0000,nypost,"China, India curb Russian oil purchases after ...",https://nypost.com/2025/10/23/world-news/china...
272,48,2025-10-23-20-01-18 +0000,nypost,Here are the corporations and donors footing t...,https://nypost.com/2025/10/23/us-news/here-are...
19,44,2025-10-24-12-24-07 +0000,nypost,Wild video shows security shoot at U-Haul truc...,https://nypost.com/2025/10/24/us-news/video-sh...
148,42,2025-10-24-04-27-12 +0000,nypost,Kash Patel skewers Stephen A. Smith for sugges...,https://nypost.com/2025/10/24/us-news/kash-pat...
335,41,2025-10-23-15-38-53 +0000,nypost,Trump pardons convicted Binance founder Changp...,https://nypost.com/2025/10/23/business/trump-p...
337,38,2025-10-23-15-32-37 +0000,nypost,Zohran Mamdani’s dad allegedly tried to silenc...,https://nypost.com/2025/10/23/world-news/zohra...
261,37,2025-10-23-20-34-00 +0000,wsj,"Norfolk Southern Third-Quarter Sales Rise, Ups...",https://www.wsj.com/business/earnings/norfolk-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
